In [19]:
import os
import tempfile
import sys
import argparse
import torch
import time
import numpy as np

from functools import partial
from line_profiler import LineProfiler
from loguru import logger
from torch import optim
from torch.nn.parallel import DistributedDataParallel as DDP
from pyscf import fci

from utils import setup_seed, Logger, ElectronInfo, Dtype, state_to_string
from utils.pyscf_helper import read_integral, interface
from utils import convert_onv, get_fock_space
from utils.det_helper import DetLUT, select_det, sort_det
from utils.distributed import get_rank
from utils.loggings import dist_print
from utils.pyscf_helper.dice_pyscf import read_dice_wf, run_shci
from vmc.ansatz import DecoderWaveFunction
from vmc.optim import VMCOptimizer
from ci_vmc.hybrid import NqsCi
from utils.ci import unpack_ucisd, ucisd_to_fci, fci_revise, CIWavefunction
from torchinfo import summary


In [20]:
device = "cpu"
atom: str = ""
bond = 1.50
for k in range(8):
    atom += f"H, 0.00, 0.00, {k * bond:.3f} ;"
integral_file = tempfile.mkstemp()[1]
sorb, nele, e_lst, fci_amp, ucisd_amp, mf = interface(
    atom, integral_file=integral_file, cisd_coeff=True,
    basis="sto-3g",
    localized_orb=False,
    localized_method="meta-lowdin",
)
logger.info(e_lst)
h1e, h2e, ci_space, ecore, sorb = read_integral(
    integral_file,
    nele,
    # save_onstate=True,
    # external_onstate="profiler/H12-1.50",
    # ##given_sorb= (nele + 2),
    device=device,
    # prefix="test-onstate",
)

System: uname_result(system='Darwin', node='imacbooks-MacBook-Pro.local', release='24.2.0', version='Darwin Kernel Version 24.2.0: Fri Oct 25 20:34:53 PDT 2024; root:xnu-11215.60.400.0.1~11/RELEASE_ARM64_T8103', machine='arm64')  Threads 1
Python 3.9.19 (main, Mar 21 2024, 12:07:41) 
[Clang 14.0.6 ]
numpy 1.26.4  scipy 1.13.0  h5py 3.11.0
Date: Wed Nov  6 13:50:40 2024
PySCF version 2.7.0
PySCF path  /Users/imacbook/anaconda3/envs/zbh/lib/python3.9/site-packages/pyscf

[ENV] PYSCF_EXT_PATH /Users/imacbook/Desktop/Research/zbh/dmrgscf:
[CONFIG] conf_file None
[INPUT] verbose = 5
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 8
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.

2024-11-06 13:50:41.768 | INFO     | __main__:<module>:13 - [-3.9954117071822868, -3.9210409123399903, -3.6719634733385096]


Integral file: /var/folders/9r/2x2s1spn3sz9vwwqt2q1h7zm0000gn/T/tmpel07jc0s
Sorb: 16
int1e: 256:1.953E-03MiB
int2e: 7260:5.539E-02MiB
Time for loading integral: 3.709E-03s


In [21]:
from utils.public_function import get_fock_space
from utils.public_function import torch_sort_onv

from libs.C_extension import get_comb_tensor, get_hij_torch, onv_to_tensor


In [22]:
fock_space = get_fock_space(sorb=sorb)
idx = torch_sort_onv(fock_space)
fock_space = fock_space[idx]
# fock_space.numpy().view(np.uint64)
fock_space_state = ((onv_to_tensor(fock_space, sorb=sorb) + 1)/2).to(torch.int64)
fock_space_state, fock_space.numpy().view(np.uint64)

(tensor([[0, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         ...,
         [1, 0, 1,  ..., 1, 1, 1],
         [0, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 array([[    0],
        [    1],
        [    2],
        ...,
        [65533],
        [65534],
        [65535]], dtype=uint64))

In [23]:
dim = fock_space.size(0)

In [ ]:
Ham = torch.zeros(dim, dim, dtype=torch.double)
for i in range(0, fock_space.shape[0]):
    x = fock_space_state[i]
    alpha = x[::2].sum().item()
    beta = x[1::2].sum().item()
    nele = alpha + beta
    comb = get_comb_tensor(fock_space[i].reshape(1, -1), sorb, nele, alpha, beta, True)[0]
    # print(comb.shape)
    # print(comb.squeeze(0))
    hij = get_hij_torch(fock_space[i].reshape(1, -1), comb.squeeze(0),h1e, h2e, sorb, nele)
    # print(hij)
    Ham[i][comb.view(torch.int64).flatten()] = hij
x = torch.linalg.eigh(Ham)
e_fci = e_lst[0]
assert abs(x[0][0].item() + ecore - e_fci) < 1.0e-10

In [ ]:
# Ham_cuda = torch.zeros(dim, dim, dtype=torch.double, device="cuda")
# fock_space_cuda = fock_space.to("cuda")
# fock_space_state_cuda = fock_space_state.to("cuda")
# h1e_cuda = h1e.to("cuda")
# h2e_cuda = h2e.to("cuda")

# for i in range(0, fock_space.shape[0]):
#     x = fock_space_state_cuda[i]
#     alpha = x[::2].sum().item()
#     beta = x[1::2].sum().item()
#     nele = alpha + beta
#     comb = get_comb_tensor(fock_space_cuda[i].reshape(1, -1), sorb, nele, alpha, beta, True)[0]
#     # print(comb.shape)
#     # print(comb.squeeze(0))
#     hij = get_hij_torch(fock_space_cuda[i].reshape(1, -1), comb.squeeze(0), h1e_cuda, h2e_cuda, sorb, nele)
#     # print(hij)
#     Ham_cuda[i][comb.view(torch.int64).flatten()] = hij
# x = torch.linalg.eigh(Ham_cuda)
# e_fci = e_lst[0]
# assert abs(x[0][0].item() + ecore - e_fci) < 1.0e-10

In [ ]:
data = []
col = []
row = []

for i in range(0, fock_space.shape[0]):
    x = fock_space_state[i]
    alpha = x[::2].sum().item()
    beta = x[1::2].sum().item()
    nele = alpha + beta
    comb = get_comb_tensor(fock_space[i].reshape(1, -1), sorb, nele, alpha, beta, True)[0]
    # print(comb.shape)
    # print(comb.squeeze(0))
    hij = get_hij_torch(fock_space[i].reshape(1, -1), comb.squeeze(0),h1e, h2e, sorb, nele)
    # print(hij)
    data.append(hij.cpu().numpy().flatten())
    col.append(np.array([i] * comb.size(1), dtype=np.int64))
    row.append(comb.cpu().numpy().view(np.uint64).flatten())

In [ ]:
data[3], col[3], row[3]

(array([-3.05149572e+00, -1.78562847e-16, -3.84988998e-02, -0.00000000e+00,
         4.84237182e-02, -5.40407971e-16,  1.78562847e-16,  3.84988998e-02,
         0.00000000e+00, -4.84237182e-02,  5.40407971e-16,  1.21864585e-01,
         0.00000000e+00,  4.44551528e-02,  0.00000000e+00,  7.38430451e-04,
         0.00000000e+00,  1.06546696e-01,  0.00000000e+00, -2.79495531e-02,
         1.00613962e-16, -4.44551528e-02,  0.00000000e+00,  8.57051168e-02,
        -1.24900090e-16, -3.11919064e-02,  0.00000000e+00,  2.79495531e-02,
         1.24900090e-16,  5.73448939e-02,  0.00000000e+00, -7.38430451e-04,
        -1.00613962e-16,  3.11919064e-02,  0.00000000e+00,  7.91410548e-02]),
 array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([   3,    6,   18,   66,  258, 1026,    9,   33,  129,  513, 2049,
          12,   24,   72,  264, 1032,   36,   48,   96,  288, 1056,  132,
         144,  192,  384, 1152,  516,  5

In [ ]:
data = np.concatenate(data)
col = np.concatenate(col)
row = np.concatenate(row)

In [ ]:
from scipy.sparse import csr_matrix
x = csr_matrix((data, (row, col)), shape=(dim, dim))

In [ ]:
type(x)

scipy.sparse._csr.csr_matrix

In [ ]:
import scipy
scipy.sparse.linalg.eigsh(x)[0][0] + ecore

-2.9955654258319484

In [ ]:
e_lst[0]

-2.995565425831943

In [ ]:
hij = get_hij_torch(ci_space, ci_space, h1e, h2e, 12, 6)

torch.linalg.eigh(hij)[0][0].item() + ecore

-2.995565425831943